# Avaliando os classificadores

## Constantes

In [1]:
rg_folder = '../RG-Dataset'
BID_folder = '../BID Dataset'

img_size = (150, 112)

## Importações

In [17]:
from image_preprocessing.filters import to_gray, decrease_noise
from image_preprocessing.rotations import rotate_90_if_vertical_rectangle

import cv2
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision, recall, f1

import os

## Funções

In [3]:
def load_image(path):
    img = cv2.imread(path)
    img_gray = to_gray(img)
    filtered_image = decrease_noise(img_gray)
    filtered_image = rotate_90_if_vertical_rectangle(filtered_image)
    resized_image = cv2.resize(filtered_image, img_size)
    return resized_image

def load_dataset():
    X = []
    y = []
    rg_files_path = f'{rg_folder}/files'
    rg_files = os.listdir(rg_files_path)

    for file_path in rg_files:
        if 'in.jpg' in file_path:
            X.append(load_image(f'{rg_files_path}/{file_path}').flatten())
            y.append(1)
    
    for folder in ['CNH_Aberta', 'CNH_Frente', 'CNH_Verso', 'CPF_Frente', 'CPF_Verso']:
        folder_path = f'{BID_folder}/{folder}'
        document_files = os.listdir(folder_path)
        for document_path in document_files:
            if 'in.jpg' in document_path:
                X.append(load_image(f'{folder_path}/{document_path}').flatten())
                y.append(0)
    
    return np.array(X), np.array(y)

def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    return {
        'acurácia': accuracy_score(y_test, y_pred),
        'precisão': precision(y_test, y_pred),
        'revocação': recall(y_test, y_pred),
        'f-medida': f1(y_test, y_pred)
    }


In [5]:
X, y = load_dataset()

X = np.array(X)
y = np.array(y)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1, stratify=y)

## Testando modelos

### SVM One Class